In [1]:
from lark import Lark, Transformer
import re
import pandas as pd

In [2]:
_FLAGS = re.MULTILINE
_LINE_TO_EXCLUDE_REGEX = re.compile(r'^(ALPHABETISCHER|WORTLIST|(GOETHE-)?ZERTIFIKAT|(A1|A2|B1|B2|C1|C2)|\d\d?)[^\n]*$', _FLAGS)
_REMOVE_LETTER_TITLE_REGEX = re.compile(r'^[A-Z]\n', _FLAGS)
_MULTIPLES_LINE_BREAKS = re.compile(r'\n\n+', _FLAGS)

def preprocess_wortliste(wortliste_text):
    
    filtered_wortliste = re.sub(_LINE_TO_EXCLUDE_REGEX, '', wortliste_text)
    filtered_wortliste = re.sub(_REMOVE_LETTER_TITLE_REGEX, '\n', filtered_wortliste)
    
    return re.sub(_MULTIPLES_LINE_BREAKS, '\n\n', filtered_wortliste).rstrip()

In [3]:
wortliste = ""
with open('../gen/goethe/goethe_a2_wortliste.txt') as wortliste_file:
    wortliste = wortliste_file.read()
    
wortliste = preprocess_wortliste(wortliste)
with open('../gen/goethe/goethe_a2_wortliste_preprocessed.txt', 'w+') as preprocessed_file:
    preprocessed_file.write(wortliste)

In [4]:
GRAMMAR = open("worteliste.lark", 'r').read()
print(GRAMMAR)

// Only detects blocks
start: (text (newline | block_end)?)*

// Intermediate variables
block_end: NEWLINE NEWLINE
newline: NEWLINE
text: TEXT

// Leaf definitions
NEWLINE: (CR? LF)
TEXT: /[^\n]+/ 

// Imports
%import common.CR
%import common.LF
%import common.WS_INLINE

// Ignores
%ignore WS_INLINE



In [7]:
transformer = Transformer()
ctr_parser = Lark(GRAMMAR, start='start', parser='lalr', transformer=transformer)
parse_tree = ctr_parser.parse(wortliste)
#print(parse_tree.children[0].children)
#for inst in parse_tree.children:

result_pair = [None, None]
texts = []
i = 0
values = {'wort': [], 'zum_beispiel': []}

for child in parse_tree.children:
    if child.data == 'text':
        texts.append(child.children[0].value)
    if child.data == 'block_end':
        result_pair[i] = '\n'.join(texts)
        i += 1
        texts = []
        if i == 2:
            values['wort'].append(result_pair[0])
            values['zum_beispiel'].append(result_pair[1])
            i = 0
            result_pair = [None, None]

pd.DataFrame(values)

,wort,zum_beispiel
0,hat angeboten,Er hat mir eine Stelle als Verkäuferin angeboten.
1,"das Angebot, -e",Heute sind Bananen im Angebot.\nStellenangebot...
2,ander-,"Willst du diesen Mantel? – Nein, ich möchte de..."
3,"ändern, \nändert,\nhat geändert",Das Wetter hat sich geändert.\nWie kann ich me...
4,anders,Anders geht das leider nicht.\nOliver ist ande...
5,"anfangen,\nfängt an,",Hier fängt die Bahnhofstraße an.\nDer Unterric...
6,hat angefangen,Würden Sie bitte anfangen?\nIch fange im Herbs...
7,"der Anfang, -e",Sie wohnt am Anfang der Straße.\nWir machen An...
8,Achtung (Sg.),Achtung an Gleis drei: Die Türen schließen. De...
9,"die Angst, -e",Hab keine Angst!\nIch habe Angst vor großen Hu...
